In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import os
import pandas as pd
import re

import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')



In [3]:
fn_EcoCyc = '../data/sRNA/EcoCyc/EcoCyc_regulatory_RNAs.csv'
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
fn_srnatarbase = os.path.join('..', 'data', 'sRNA', 'sRNATarBase', 'sRNATarBase.csv')
ecocyc = pd.read_csv(fn_EcoCyc)
rnainter = pd.read_csv(fn_RNAInter)
srnatarbase = pd.read_csv(fn_srnatarbase)
# data = pd.concat([pd.read_csv(fn_RNAInter), pd.read_csv(fn_EcoCyc)])
try:
    rnainter = rnainter.drop(columns=['Unnamed: 0'])
    srnatarbase = srnatarbase.drop(columns=['Unnamed: 0'])
except:
    pass


### Start with RNAInter

In [4]:
np.sum(rnainter['Sequence1'].isna()) + np.sum(rnainter['Sequence2'].isna())

2

In [5]:
rnainter

,level_0,index,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict,Sequence1,Sequence2
0,0,1029724,RR05384747,acnA,mRNA,Escherichia coli str. K-12 substr. MG1655,ryhB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:946724,NCBI:2847761,0.2292,Northern blot//Reporter assay,NaN,NaN,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...
1,1,1029725,RR05384845,acrZ,mRNA,Escherichia coli str. K-12 substr. MG1655,omrB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945365,NCBI:2847747,0.1778,Northern blot,Microarray,NaN,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,TGTTCTATACTTGGGTTCGACTTGGGTTAGACTTGTCTTTACTGTC...
2,2,1029732,RR05387056,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,flhD,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:945442,0.2314,Northern blot//Reporter assay,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,CTCGTGCCGAATTCGGCACGAGCGATATTTCATCAGTTATCGGTAA...
3,3,1029733,RR05387057,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,rpoS,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:947210,0.1996,Northern blot//RACE//RT-PCR//Beta-galactosidas...,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...
4,4,1029734,RR05387126,argR,mRNA,Escherichia coli str. K-12 substr. MG1655,dsrA,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:947861,NCBI:946470,0.2139,Primer extension assay,NaN,NaN,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,349,8031771,RR05460721,ryhB,sRNA,Escherichia coli str. K-12 substr. MG1655,ynfF,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847761,NCBI:945268,0.1778,Northern blot,Microarray,NaN,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,ACTTACTACTGTGCCTTTAGGGTGGGGGGATACACCGTTTCCGCCG...
350,350,8031772,RR05460722,sdsR,sRNA,Escherichia coli str. K-12 substr. MG1655,tolC,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847772,NCBI:947521,0.3292,Northern blot//Western blot,NaN,NaN,CTACCTCAACACCCAACAATCGTTGATTGAAAACATCTTCTATCGC...,ATGAAGAAATTGCTCCCCATTCTTATCGGCCTGAGCCTTTCTGGGT...
351,351,8031773,RR05460723,sgrS,sRNA,Escherichia coli str. K-12 substr. MG1655,sodB,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:4056038,NCBI:944953,0.1983,Northern blot,NaN,NaN,CCAGAACATTGGCTTTCAAAAGTTTTTTAATTGATGGAAACTCTTT...,GAAGAGAAAAGAAAAAAAGACCTTATTTGCTTATCCTGCATCTCTC...
352,352,8031774,RR05460724,sgrS,sRNA,Escherichia coli str. K-12 substr. MG1655,yigL,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:4056038,NCBI:2847768,0.3273,Northern blot//Reporter assay,NaN,NaN,CCAGAACATTGGCTTTCAAAAGTTTTTTAATTGATGGAAACTCTTT...,TGCCGCTGACGAGGGAGAGGATGAGGATTGCGAGCCAGAGCATCCA...


In [6]:
data = pd.DataFrame()
rnainter['Database'] = 'RNAInter'
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor1.Symbol', 'Raw_ID1', 'Category1', 'Interactor1.Symbol', 'Sequence1', 'Database']):
    data[n] = rnainter.drop_duplicates(subset=['Interactor1.Symbol'])[o]
data2 = pd.DataFrame()
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2', 'Database']):
    data2[n] = rnainter.drop_duplicates(subset=['Interactor2.Symbol'])[o]

# data = pd.concat([data, rnainter.drop_duplicates(subset=['Interactor2.Symbol']).reset_index()[['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Sequence2']].rename(
#     columns={j: i for i, j in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2'])})], axis=0)
data = pd.concat([data, data2])
data['Alias'] = ''

data = data[~data['Name'].isna()]
data = data.drop_duplicates(subset=['Name'])

In [7]:
data['Symbol'].value_counts()


Symbol
acnA    1
phoP    1
sibC    1
sibB    1
sibA    1
       ..
luxS    1
malK    1
manX    1
manY    1
symR    1
Name: count, Length: 276, dtype: int64

### Now EcoCyc

In [8]:
ecocyc.head()

,Unnamed: 0,Ontology - all instances of class from My Favorites,Matches,Common-Name,GO-Terms,Names,Object ID,Gene,Sequence - DNA sequence,Right-End-Position,Left-End-Position,Product,Accession-1,Accession-2,Direction
0,0,6S-RNA,Regulatory-RNAs,6S RNA,NaN,6S RNA // SsrS // Ssr,6S-RNA,EG30099,ATTTCTCTGAGATGTTCGCAAGCGGGCCAGTCCCCTGAGCCGATAT...,3056165,3055983,6S-RNA,b2911,ECK2906,+
1,1,RDLD-RNA,Regulatory-RNAs,antisense regulatory RNA RdlD,GO:2000766 // GO:0030371,antisense regulatory RNA RdlD // RdlD // RdlD ...,RDLD-RNA,G0-9042,GTCTAGAGTCAAGATTAGCCCCCGTGGTGTTGTCAGGTGCATACCT...,3700199,3700136,RDLD-RNA,b4454,ECK3525,+
2,2,RNA0-441,Regulatory-RNAs,arfA-derived small regulatory RNA,GO:0006970 // GO:0003729,<i>arfA</i>-derived small regulatory RNA,RNA0-441,G0-17130,AAGAAAGTAAATCACTTTTTTACCACTGGTCTTCTGCTTTCAGGCG...,3438484,3438431,RNA0-441,b4846,ECK4703,-
3,3,RNA0-409,Regulatory-RNAs,e14 prophage; small regulatory RNA C0293,GO:0035194 // GO:0040033 // GO:0003729,e14 prophage; small regulatory RNA C0293 // C0293,RNA0-409,G0-8897,TACTCACCCGGGACTCGCCAGGGGACAGCCAACAGGCATTGGGTGC...,1196782,1196711,RNA0-409,b4806,ECK4663,+
4,4,RNA0-162,Regulatory-RNAs,putative antisense regulatory RNA RdlB,GO:1902201,putative antisense regulatory RNA RdlB // RdlB,RNA0-162,G0-9606,GTCTGGTTTCAAGATTAGCCCCCGTTCTGTTGTCAGGTTTTACCTC...,1269923,1269858,RNA0-162,b4422,ECK1207,+


In [9]:
ecocyc['Category'] = 'sRNA'
ecocyc['Database'] = 'EcoCyc'
ecocyc = ecocyc.drop_duplicates(subset=['Common-Name'])

ecocyc['Symbol'] = ecocyc['Common-Name']
ecocyc['ID'] = 'chromosome:NC_000913.3, Gene ID:' # E. coli NCBI accession
ecocyc['Genomic position'] = list(map(lambda l, r: '..'.join([l, r]), ecocyc['Left-End-Position'].apply(str), ecocyc['Right-End-Position'].apply(str)))
ecocyc['Alias'] = list(map(lambda x: ';'.join('//'.join([str(i) for i in x]).split('//')).replace(' ; ', ';'), ecocyc[['Names', 'Accession-1', 'Accession-2']].to_numpy()))
data = pd.concat([data, ecocyc.rename(columns={j: i for i, j in zip(['Name', 'Sequence', 'Strand'], ['Common-Name', 'Sequence - DNA sequence', 'Direction'])})[
    ['Name', 'ID', 'Category', 'Symbol', 'Alias', 'Sequence', 'Database', 'Genomic position', 'Strand']]], axis=0)
strand = {'+': 'forward', '-': 'reverse'}
data.loc[~data['Strand'].isna(), 'Strand'] = data[~data['Strand'].isna()]['Strand'].apply(lambda x: strand[x] if x else x)


### Finally sRNATarBase

In [10]:
srnatarbase.loc[srnatarbase['sRNA Type'].isna(), 'sRNA Type'] = 'sRNA'
srnatarbase.loc[srnatarbase['Target Type'].isna(), 'Target Type'] = 'mRNA'
srnatarbase['sRNA Type'].unique()

array(['trans-encoded antisense RNA', 'protein binding small RNA', 'sRNA'],
      dtype=object)

In [11]:
srnatarbase_merged = pd.DataFrame()
srnatarbase_merged['Name'] = np.concatenate(
    [srnatarbase['sRNA'].unique(), srnatarbase['Target'].unique()], axis=0)
srnatarbase_merged['ID'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA ID'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target ID'].iloc[0])
srnatarbase_merged['Alias'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA Alias'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Alias'].iloc[0])
srnatarbase_merged['Genomic position'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA Genome Position'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Genome Position'].iloc[0])
srnatarbase_merged['Strand'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA Strand'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Strand'].iloc[0])
srnatarbase_merged['Category'] = srnatarbase_merged['Name'].apply(
    lambda x: 'sRNA' if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Type'].iloc[0])
srnatarbase_merged['Symbol'] = srnatarbase_merged['Name']
srnatarbase_merged['Sequence'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA Sequence'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Sequence'].iloc[0])
srnatarbase_merged['Database'] = 'sRNATarBase'
srnatarbase_merged

,Name,ID,Alias,Genomic position,Strand,Category,Symbol,Sequence,Database
0,dsrA,"chromosome:NC_000913.3, Gene ID:946470",ECK1952; IS095; JWR0036,2025227..2025313,reverse,sRNA,dsrA,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,sRNATarBase
1,micF,"chromosome:NC_000913.3, Gene ID:2847742",ECK2208; IS113; JWR0043; stc,2313084..2313176,forward,sRNA,micF,GCTATCATCATTAACTTTATTTATTACCGTCATTCATTTCTGAATG...,sRNATarBase
2,micC,"chromosome:NC_000913.3, Gene ID:2847713",ECK1373; IS063; JWR0234; tke8,1437121..1437229,forward,sRNA,micC,GTTATATGCCTTTATTGTCACAGATTTTATTTTCTGTTGGGCCATT...,sRNATarBase
3,oxyS,"chromosome:NC_000913.3, Gene ID:2847701",ECK3952; JWR0098,4158285..4158394,reverse,sRNA,oxyS,GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,sRNATarBase
4,istR,"chromosome:NC_000913.3, Gene ID:5061525",ECK4425; psr19; psrA19,3853118..3853257,reverse,sRNA,istR,GCACTAAATACGTCAAAATTCGTGCCGAAATTGCGCGTTCTGCGCG...,sRNATarBase
...,...,...,...,...,...,...,...,...,...
319,msrA,"chromosome:NC_000913.3, Gene ID:948734",ECK4215; JW4178; pms; pmsR,4441538..4442176,reverse,mRNA,msrA,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase
320,hdeA,"chromosome:NC_000913.3, Gene ID:948025",ECK3494; JW3478; yhhC; yhiB,3656408..3656740,reverse,mRNA,hdeA,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase
321,hdeB,"chromosome:NC_000913.3, Gene ID:948026",ECK3493; JW5669; yhhD; yhiC,3655966..3656292,reverse,mRNA,hdeB,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase
322,fliC,"chromosome:NC_002695.1, Gene ID:",ECs2662,2624379..2626136,reverse,mRNA,fliC,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase


In [12]:
data = pd.concat([data, srnatarbase_merged], axis=0)
data = data.reset_index().drop(columns='index')

# Delete duplicates responsibly

In [13]:
data['Name overlaps'] = list(map(lambda n: [n2 for n2, a in zip(data['Name'], data['Alias']) if (n.lower() in str(a).lower()) or (n.lower() in n2.lower())], data['Name']))

In [14]:
doubles = []
for i, r in data.iterrows():
    if len(r['Name overlaps']) > 1:
        if len(data[data['Name'].isin(r['Name overlaps'])]['Sequence'].unique()) == 1:
            doubles.append(r['Name overlaps'])
            
for d in doubles:
    shortest = np.argmin(list(map(len, d)))
    for drop in [i for i in d if i != d[shortest]]:
        print('Removing', drop)
        data = data.drop(index=data[data['Name'] == drop].index[0])


Removing small regulatory RNA GlmY
Removing small regulatory RNA CsrB
Removing 6S RNA


In [15]:
# doubles = data['Symbol'].value_counts()[data['Symbol'].value_counts() > 1].index.to_list()
doubles = set(data[data['Name overlaps'].apply(len) > 1]['Name'])
len(doubles)

278

In [16]:
data[data['Name'] == 'micA']

,Name,ID,Category,Symbol,Sequence,Database,Alias,Genomic position,Strand,Name overlaps
108,micA,NCBI:2847697,sRNA,micA,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,RNAInter,,NaN,NaN,"[micA, small regulatory RNA MicA, MicA]"


In [17]:

for d in doubles:
    
    if len(data[data['Symbol'] == d]['Sequence'].unique()) == 1:
        print('Removing duplicate of ', d, data[data['Symbol'] == d]['Name'].unique())
        data = data.drop(index=data[(data['Symbol'] == d) & (data['Database'] != 'EcoCyc')].index[0])
    else:
        print('Did not remove', d)
        # print(data[data['Symbol'] == d]['Sequence'].values)
    
data = data[~data['Sequence'].isna()]
data
# data[data['Symbol'] == d].index[0]

Did not remove yfeK
Did not remove rrrD
Removing duplicate of  malK ['malK']
Removing duplicate of  cysE ['cysE']
Did not remove yccS
Did not remove cydD
Did not remove rdlC
Did not remove shiA
Did not remove ldrB
Did not remove gmhB
Did not remove tolB
Removing duplicate of  MicA ['MicA']
Did not remove acrZ
Removing duplicate of  pmbA ['pmbA']
Removing duplicate of  hyuA ['hyuA']
Did not remove folP
Removing duplicate of  csgB ['csgB']
Did not remove mipA
Did not remove yeeE
Removing duplicate of  uhpT ['uhpT']
Did not remove sokB
Did not remove moaD
Did not remove sodB
Did not remove fdoI
Removing duplicate of  rydC ['rydC']
Did not remove rdlA
Did not remove ompF
Did not remove clpB
Did not remove xylH
Removing duplicate of  ycfL ['ycfL']
Removing duplicate of  ygiM ['ygiM']
Did not remove msrB
Removing duplicate of  marA ['marA']
Did not remove citA
Removing duplicate of  narK ['narK']
Did not remove symR
Did not remove arcZ
Removing duplicate of  mraZ ['mraZ']
Did not remove erpA

,Name,ID,Category,Symbol,Sequence,Database,Alias,Genomic position,Strand,Name overlaps
0,acnA,NCBI:946724,mRNA,acnA,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,RNAInter,,NaN,NaN,"[acnA, acnA]"
1,acrZ,NCBI:945365,mRNA,acrZ,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,RNAInter,,NaN,NaN,"[acrZ, acrZ]"
2,arcZ,NCBI:2847690,sRNA,arcZ,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,RNAInter,,NaN,NaN,"[arcZ, small regulatory RNA ArcZ, arcZ]"
3,argR,NCBI:947861,mRNA,argR,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,RNAInter,,NaN,NaN,"[argR, argR]"
4,arrS,NCBI:11115378,sRNA,arrS,CTCTCCCTCTCTTTCTCTCTTCTCCGCGGCGATACGATCCGAGATG...,RNAInter,,NaN,NaN,"[arrS, small regulatory RNA ArrS, arrS]"
...,...,...,...,...,...,...,...,...,...,...
670,msrA,"chromosome:NC_000913.3, Gene ID:948734",mRNA,msrA,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase,ECK4215; JW4178; pms; pmsR,4441538..4442176,reverse,"[msrA, msrA]"
671,hdeA,"chromosome:NC_000913.3, Gene ID:948025",mRNA,hdeA,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase,ECK3494; JW3478; yhhC; yhiB,3656408..3656740,reverse,"[hdeA, hdeA]"
672,hdeB,"chromosome:NC_000913.3, Gene ID:948026",mRNA,hdeB,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase,ECK3493; JW5669; yhhD; yhiC,3655966..3656292,reverse,"[hdeB, hdeB]"
673,fliC,"chromosome:NC_002695.1, Gene ID:",mRNA,fliC,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase,ECs2662,2624379..2626136,reverse,[fliC]


# Surrounding sequences

Since some of the interactions between sRNAs and mRNAs are upstream of the mRNA transcription or translation start site.

In [18]:
from Bio import Entrez
from urllib.error import HTTPError


Entrez.email = "olivia.gallup@gmail.com"  # Add your email address
Entrez.api_key = "8167226cf01abaa985a1c23d9b5283d22208"  # Add your NCBI API key


def is_complement(gene_id):
    handle = Entrez.efetch(db="gene", id=gene_id, rettype="txt", retmode="gb")
    record = handle.read()
    if 'complement' in record:
        return True
    return False


def get_seq_from_genome(gene_id: int, thresh: int, before: bool, dna_start: int = None, dna_end: int = None, nuc_id: str = None, strand: str = None):

    try:
        if not strand:
            is_comp = is_complement(gene_id)
        else:
            is_comp = True if strand == 'reverse' else False
        if not dna_start and (not dna_end):
            handle = Entrez.efetch(db="gene", id=gene_id,
                                rettype="fasta", retmode="xml")
            record = Entrez.read(handle)
            if not nuc_id:
                nuc_id = record[0]['Entrezgene_locus'][0]['Gene-commentary_accession'] + \
                    '.' + record[0]['Entrezgene_locus'][0]['Gene-commentary_version']

            if (before and not(is_comp)) or (is_comp and not(before)):
                dna_start = int(record[0]['Entrezgene_locus'][0]['Gene-commentary_seqs']
                                [0]['Seq-loc_int']['Seq-interval']['Seq-interval_from']) - thresh
                dna_end = int(record[0]['Entrezgene_locus'][0]['Gene-commentary_seqs']
                            [0]['Seq-loc_int']['Seq-interval']['Seq-interval_from'])
            if (is_comp and before) or (not(is_comp) and not(before)):
                dna_start = int(record[0]['Entrezgene_locus'][0]['Gene-commentary_seqs']
                                [0]['Seq-loc_int']['Seq-interval']['Seq-interval_to'])
                dna_end = int(record[0]['Entrezgene_locus'][0]['Gene-commentary_seqs']
                            [0]['Seq-loc_int']['Seq-interval']['Seq-interval_to']) + thresh

        search_handle = Entrez.efetch(db="nuccore", id=nuc_id, retmode="text", rettype='gb',
                                      strand="complement" if is_comp else '', seq_start=int(dna_start), seq_stop=int(dna_end))
        # dna_sequence = SeqIO.read(search_handle, "fasta").seq
        r = search_handle.read()
        if r:
            if len(r.split('>')) == 1 or (len(r.split('>')[1].split('\n')) <= 1):
                dna_sequence = ''
            # dna_sequence = ''.join(r.split('>')[1].split('\n')[1:])
            dna_sequence = re.sub('[^atcgACTG]', '', (r.split('ORIGIN')[-1]))
        else:
            dna_sequence = ''
        return dna_sequence
    except HTTPError:
        return ''


def get_seq_info(gene_id: int, thresh: int = 200, before: bool = True, dna_start: int = None, dna_end: int = None, nuc_id: str = None, strand: str = None):
    try:
        handle = Entrez.efetch(db="nucleotide", id=gene_id,
                               rettype="fasta", retmode="text")
        fast = handle.read()
        seq = ''.join(fast.split('\n')[1:])
        if not seq or ('N' in seq):
            seq = get_seq_from_genome(gene_id, thresh, before, dna_start, dna_end, nuc_id, strand)
        return seq
    except HTTPError:
        seq = get_seq_from_genome(gene_id, thresh, before, dna_start, dna_end, nuc_id, strand)
        return seq

In [19]:
data['Raw ID'] = data['ID'].str.split(':').apply(lambda x: x[-1])

In [20]:
from Bio import Entrez

def find_gene_accession(species_number, gene_name):
    # Provide your email to NCBI
    Entrez.email = "your.email@example.com"

    try:
        # Search for gene information related to the specified species number
        search_term = f"{species_number}[Accession] AND {gene_name}"
        handle = Entrez.esearch(db="nucleotide", term=search_term)
        record = Entrez.read(handle)
        handle.close()

        # Retrieve the first accession number found
        if record["Count"] != "0":
            accession_number = record["IdList"][0]
            return int(accession_number)
        else:
            return None

    except Exception as e:
        return f"An error occurred: {e}"


print(f'Total:', len(data[(data['Raw ID'] == '') | (data['Raw ID'].isna())]), '\n')
for i, interactors in data[(data['Raw ID'] == '') | (data['Raw ID'].isna())].iterrows():
    gene_name = interactors['Name']
    species_number = interactors['ID'].split(',')[0].split(':')[-1]
    accession_number = find_gene_accession(species_number, gene_name)
    if type(accession_number) != int:
        alias = data[data['Name'] == gene_name]['Alias'].iloc[0]
        if type(alias) == str:
            gene_names = [ii.strip(' ') for ii in alias.split(';') if ii.strip(' ')]
            cand_accession_numbers = []
            for g in gene_names:
                accession_number = find_gene_accession(species_number, gene_name)
                if type(accession_number) != int:
                    cand_accession_numbers.append(accession_number)
            if not all(element == cand_accession_numbers[0] for element in cand_accession_numbers) or (type(cand_accession_numbers[0]) == str):
                print(f'Warning: the accession number for {gene_name} found through aliases {gene_names} did not match: {cand_accession_numbers}')
            accession_number = cand_accession_numbers[0]
    
    data['Raw ID'].loc[i] = accession_number


Total: 87 



In [21]:
data[(data['Raw ID'] == '') | (data['Raw ID'].isna())]

,Name,ID,Category,Symbol,Sequence,Database,Alias,Genomic position,Strand,Name overlaps,Raw ID
285,small antisense RNA aMEF,"chromosome:NC_000913.3, Gene ID:",sRNA,small antisense RNA aMEF,CAAATCAGATCGCCCATATCGGGTACGTATCGGCTTACCATTACCA...,EcoCyc,small antisense RNA aMEF;aMEF;b4844;ECK4701,2911051..2911450,forward,[small antisense RNA aMEF],None
286,small antisense RNA AsCRP,"chromosome:NC_000913.3, Gene ID:",sRNA,small antisense RNA AsCRP,CCGCTTTTTCACCCTGGTGAATAAGCGTGCTCTTGGATGGGTACTT...,EcoCyc,small antisense RNA AsCRP;b4841;ECK4698,3485953..3486231,reverse,[small antisense RNA AsCRP],None
287,small antisense RNA AsflhD,"chromosome:NC_000913.3, Gene ID:",sRNA,small antisense RNA AsflhD,TCAGCAACTCGGAGGTATGCATTATTCCCACCCAGAATAACCAACT...,EcoCyc,small antisense RNA AsflhD;AsflhD;b4839;ECK4696,1978176..1978395,forward,[small antisense RNA AsflhD],None
288,small antisense RNA AsphoP,"chromosome:NC_000913.3, Gene ID:",sRNA,small antisense RNA AsphoP,CACCGGCACTTAATACTTCGACTTTGTCCTGCCAGCTTTCACGGGC...,EcoCyc,small antisense RNA AsphoP;b4843;ECK4700,1190165..1190508,forward,[small antisense RNA AsphoP],None
289,small antisense RNA StfZ,"chromosome:NC_000913.3, Gene ID:",sRNA,small antisense RNA StfZ,GGTGATACCGCTACCGATTTGAATCGTCTGTCCAACCGCTGTTTTA...,EcoCyc,small antisense RNA StfZ;StfZ;b4840;ECK4697,104948..105499,reverse,[small antisense RNA StfZ],None
298,small regulatory RNA AzuR,"chromosome:NC_000913.3, Gene ID:",sRNA,small regulatory RNA AzuR,AAAGACAATAACACCTGTATAACAAATGGTCGGAGTGCCGCGATGA...,EcoCyc,small regulatory RNA AzuR;AzuR;b4434;ECK1902,1987843..1988001,reverse,[small regulatory RNA AzuR],None
314,small regulatory RNA GlnZ,"chromosome:NC_000913.3, Gene ID:",sRNA,small regulatory RNA GlnZ,TTTTAGTTGCCGTGGAAACTTTTCGCCTGTCTCTGGCAGGCCTGGG...,EcoCyc,small regulatory RNA GlnZ;glnA 3<sup>A</sup> U...,4056428..4056621,reverse,[small regulatory RNA GlnZ],None
324,small regulatory RNA Och5,"chromosome:NC_000913.3, Gene ID:",sRNA,small regulatory RNA Och5,ATCCTTGTCCGCTTAACCGTTTCAGTCAGCCTATCCTTGATGAAAC...,EcoCyc,small regulatory RNA Och5;Och5;nan;nan,3067187..3067344,forward,[small regulatory RNA Och5],None
363,Spot_42,"chromosome:NC_000913.3, Gene ID:",sRNA,Spot_42,GTAGGGTACAGAGGTAAGATGTTCTATCTTTCAGACCTTTTACTTC...,sRNATarBase,spf ; ECK3856; IS197; JWR0097;,4049899..4050007,forward,[Spot_42],None
390,OrzO,"chromosome:NC_002695.1, Gene ID:",sRNA,OrzO,GTTGGTACGAAATGTTGCACAATCAGCAAGTTAGCGTCACTGTAGT...,sRNATarBase,NaN,2883467..2883547,forward,[OrzO],None


In [22]:
seqbefore = {n: get_seq_info(None, thresh=200, before=True, dna_start=p.split('..')[0], dna_end=p.split('..')[1], nuc_id=i.split(':')[1].split(',')[0], strand=s) for n, p, i, s in list(data[data['Raw ID'].isna()].groupby(['Name', 'Genomic position', 'ID', 'Strand']).apply(str).index)}
seqafter = {n: get_seq_info(None, thresh=200, before=False, dna_start=p.split('..')[0], dna_end=p.split('..')[1], nuc_id=i.split(':')[1].split(',')[0], strand=s) for n, p, i, s in list(data[data['Raw ID'].isna()].groupby(['Name', 'Genomic position', 'ID', 'Strand']).apply(str).index)}

In [23]:
seqbefore.update({n: get_seq_info(str(k), thresh=200, before=True) for n, k in list(data[~data['Raw ID'].isna()].groupby(['Name', 'Raw ID']).apply(str).index)})
seqafter.update({n: get_seq_info(str(k), thresh=200, before=False) for n, k in list(data[~data['Raw ID'].isna()].groupby(['Name', 'Raw ID']).apply(str).index)})


In [24]:
data['Sequence before'] = data['Name'].apply(lambda x: seqbefore[x])
data['Sequence after'] = data['Name'].apply(lambda x: seqafter[x])

In [37]:
while len(data[data['Sequence before'] == '']) > 0:
    for i, r in data[data['Sequence before'] == ''].iterrows():
        data.loc[i, 'Sequence before'] = get_seq_info(str(r['Raw ID']), thresh=200, before=True)

In [38]:
data[data['Sequence before'] == '']

,Name,ID,Category,Symbol,Sequence,Database,Alias,Genomic position,Strand,Name overlaps,Raw ID,Sequence before,Sequence after


In [36]:
# get_seq_info(str(946693), thresh=200, before=True)

'cgaagggctggtgcgaatcaaccgtcagggcgatgacctgcacattgaaacgcaaaacgttgcgccaccggacagccgtattgagctgatttccagcagcgaagctgactggaatgccttgcagagcgcgctgttgaagcttcgtttagcgactaccgcgtaaggttgcctgcgtttttcagtaagataattagagaaaat'

In [27]:
# get_seq_info(str(948588), thresh=200, before=False)

'cggcggtatccatataagtacggatttgactgaacgtccccaggcggctgtcggcaataataatgccgagcgccaccagcaccgccagaataaggcgaatctgtagcgacgggccacggctaaaaattggcttcataagttatgcgtattctcgtatcagaccaggcagggtaaacagacacttcccctgcctgcatccga'

# Save

In [28]:
data.columns

Index(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database', 'Alias',
       'Genomic position', 'Strand', 'Name overlaps', 'Raw ID',
       'Sequence before', 'Sequence after'],
      dtype='object')

In [29]:
data[['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database', 'Alias',
      'Genomic position', 'Strand',
      'Sequence before', 'Sequence after']].to_csv('../data/sRNA/merged_EcoCyc_RNAInter_sRNATarBase.csv')